In [1]:
import sqlite3
conn = sqlite3.connect('airline2.db')

In [2]:
import pandas as pd

airports = pd.read_csv("airports.csv")
carriers = pd.read_csv("carriers.csv")
airplanes = pd.read_csv("planedata.csv")
df1 = pd.read_csv("2000.csv", encoding='latin-1')
df2 = pd.read_csv("2001.csv", encoding='latin-1')
df3 = pd.read_csv("2002.csv", encoding='latin-1')
df4 = pd.read_csv("2003.csv", encoding='latin-1')
df5 = pd.read_csv("2004.csv", encoding='latin-1')
df6 = pd.read_csv("2005.csv", encoding='latin-1')


C:\Users\e2u\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
ontime = pd.concat([df1, df2, df3, df4, df5, df6])

In [4]:
ontime.tail

<bound method NDFrame.tail of          Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime  ArrTime  \
0        2000      1          28          5   1647.0        1647   1906.0   
1        2000      1          29          6   1648.0        1647   1939.0   
2        2000      1          30          7      NaN        1647      NaN   
3        2000      1          31          1   1645.0        1647   1852.0   
4        2000      1           1          6    842.0         846   1057.0   
...       ...    ...         ...        ...      ...         ...      ...   
7140591  2005     12          22          4   1652.0        1655   1815.0   
7140592  2005     12          22          4   1825.0        1825   1858.0   
7140593  2005     12          22          4   1507.0        1511   1641.0   
7140594  2005     12          22          4    924.0         925   1056.0   
7140595  2005     12          22          4   1345.0        1344   1621.0   

         CRSArrTime UniqueCarrier  FlightNum 

In [5]:
airports.to_sql('airports2', con = conn, index = False) 
carriers.to_sql('carriers2', con = conn, index = False)
airplanes.to_sql('planedata2', con = conn, index = False)
ontime.to_sql('ontime2', con = conn, index = False) 

In [6]:
conn.commit()

In [7]:
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()

[('airports2',), ('carriers2',), ('planedata2',), ('ontime2',)]

In [9]:
# City with highest number of inbound flights (excluding cancelled flights) = Chicago

q1 = c.execute('''
SELECT airports2.city as city, COUNT(*) AS total
FROM airports2  JOIN ontime2 ON ontime2.dest = airports2.iata
WHERE ontime2.Cancelled = 0
GROUP BY airports2.city
ORDER BY total DESC
''').fetchall()

pd.DataFrame(q1)

,0,1
0,Chicago,2388365
1,Atlanta,1912370
2,Dallas-Fort Worth,1781833
3,Houston,1304407
4,Los Angeles,1279514
...,...,...
282,Provo,3
283,Cheyenne,2
284,Scottsbluff,1
285,Roswell,1


In [12]:
# airplane that has the lowest avg. dep. delay (excl. cancelled and diverted) = 737-2Y5

q3 = c.execute('''
SELECT model AS model, AVG(ontime2.DepDelay) AS avg_delay
FROM planedata2 JOIN ontime2 USING(tailnum)
WHERE ontime2.Cancelled = 0 AND ontime2.Diverted = 0 AND ontime2.DepDelay > 0
GROUP BY model
ORDER BY avg_delay
 ''').fetchall()

pd.DataFrame(q3)

,0,1
0,737-2Y5,7.022026
1,737-282,8.433566
2,737-230,10.458647
3,737-282C,11.765766
4,767-324,15.788051
...,...,...
141,ATR 72-212,38.267365
142,ATR-72-212,38.699195
143,717-200,38.737105
144,ERJ 190-100 IGW,41.723958


In [13]:
# Company with the highest number of cancelled flights = Delta Air Lines

q4 = c.execute('''
SELECT carriers2.Description AS carrier2, COUNT(*) AS total
FROM carriers2 JOIN ontime2 ON ontime2.UniqueCarrier = carriers2.Code
WHERE ontime2.Cancelled = 1 AND carriers2.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
GROUP BY carriers2.Description
ORDER BY total DESC 
 ''').fetchall()

pd.DataFrame(q4)

,0,1
0,Delta Air Lines Inc.,107851
1,American Airlines Inc.,105762
2,United Air Lines Inc.,102066


### q2 below is assumed to be q4 mentioned in the assignment.
__
##### It's also the answer to question 4 of the assignment (simplified version of the code)

In [14]:
# This is basicly just q4 (line 10) divided with q5 (the code below):

q5 = c.execute('''
SELECT carriers2.Description AS carrier2, COUNT(*) AS total
FROM carriers2 JOIN ontime2 ON ontime2.UniqueCarrier = carriers2.Code
WHERE carriers2.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
GROUP BY carriers2.Description
ORDER BY total DESC 
 ''').fetchall()
z4 = pd.DataFrame(q4)
z5 = pd.DataFrame(q5)
q2 = z4[1]/z5[1]

q2

0    0.024082
1    0.023841
2    0.027924
Name: 1, dtype: float64

In [ ]:
# The answer is no. 2 = United Air Lines Inc with approx. 2.8%